In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!nvidia-smi

Wed Oct 25 11:24:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# Load model directly
import torch
from transformers import AutoProcessor, AutoModelForTextToWaveform

model_dir = '/content/drive/MyDrive/GitHub/TTS/bark-small'

processor = AutoProcessor.from_pretrained(model_dir)
model = AutoModelForTextToWaveform.from_pretrained(model_dir)

# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    model = model.to('cuda')  # 모델을 GPU로 이동

HFValidationError: ignored

In [ ]:
inputs = processor(
    text=["Hello, This is Bark — test . And, uh — and I like pizza. [laughs] But I also have other interests such as playing tic tac toe."],
    return_tensors="pt",
)

In [ ]:
# 입력 텐서를 GPU로 이동
if torch.cuda.is_available():
    inputs = {name: tensor.to('cuda') for name, tensor in inputs.items()}

# 모델 실행
with torch.no_grad():  # 그래디언트 계산 비활성화
    speech_values = model.generate(**inputs, do_sample=True)

In [ ]:
speech_values = model.generate(**inputs, do_sample=True)

In [ ]:
from IPython.display import Audio

sampling_rate = model.generation_config.sample_rate
Audio(speech_values.cpu().numpy().squeeze(), rate=sampling_rate)

In [ ]:
import os

# 디렉토리 경로
dir_path = '/content/drive/MyDrive/GitHub/TTS/bark-small/Output'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

# 이제 .wav 파일 저장
write_wav(os.path.join(dir_path, "bark-test.wav"), sampling_rate, audio_data)